In [1]:
from data.delegate import ASGLoadDelegate

In [2]:
from trainers import GlobalParams
from data.collate_fns import convert_batch_sparse_matrix_collate_fn

In [10]:
from thexp import globs 
import os

In [3]:
params = GlobalParams()
# params.dim_fts = [2048,512]

In [4]:
from data.dataxy import mscoco

In [5]:
datas =mscoco('test',set_root='asg2cap/ControllableImageCaption/MSCOCO')

In [6]:
region_ids, json_fs, hdf5_fs, mp_ids, mp_fts = datas

In [8]:
cocoimg_root = '/home/yanghaozhe/datasets/COCO/2014/'

In [11]:
json_fn = json_fs[2].strip('.json')

img_path = os.path.join(cocoimg_root,'{}.jpg'.format('/'.join(json_fn.split('/')[-2:])))

In [35]:
kk = dict(next(iter(loader)))

In [49]:
import numpy as np

np.array(kk['caption_ids']).astype(np.long)

array([  0,   3,  36,  79,   4,  29,  64,   9,   3, 105,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1])

In [15]:
kk['mp_fts'].shape

torch.Size([64, 2048])

In [17]:
kk['rel_edges'].shape

(64, 6, 10, 10)

In [4]:
from trainers.mixin.models import AsgModelMixin
from trainers.mixin.datasets import BaseSupDatasetMixin

In [5]:
from trainers.asg2cap import BaseTrainer

In [6]:
trainer = BaseTrainer(params)

add output channel on /home/yanghaozhe/.thexp/experiments/asg2cap.e0/asg2cap.trainers/0001.a1cefe81/l.210205164028.0.log
/opt/conda/lib/python3.7/site-packages/xpython_launcher.py -f /root/.local/share/jupyter/runtime/kernel-acbcfbcb-0ee2-4a71-ac64-a4d4d35cd6d4.json | 
21-02-05 16:40:28 | Exp BaseDir | /home/yanghaozhe/.thexp/experiments/asg2cap.e0/asg2cap.trainers
21-02-05 16:40:28 | Exp Trainer | BaseTrainer
21-02-05 16:40:28 | Exp Params
GlobalParams[('epoch', 100),
 ('eidx', 1),
 ('idx', 0),
 ('global_step', 0),
 ('device', 'cuda:0'),
 ('device_ids', []),
 ('dataset', 'mscoco'),
 ('architecture', 'rgcn.flow.memory'),
 ('optim',
  OptimParams([('args', attr([('lr', 0.0002), ('weight_decay', 0)])),
               ('name', 'Adam')])),
 ('git_commit', True),
 ('tmp_dir', None),
 ('distributed', False),
 ('world_size', -1),
 ('local_rank', -1),
 ('init_method', 'env://'),
 ('attn_encoder', 'role_rgcn'),
 ('decoder', 'memory_flow'),
 ('n_classes', 10),
 ('topk', (1, 2, 3, 4)),
 ('batch_s

In [27]:
batch = trainer.test_dataloader.choice_batch()

In [9]:
from thexp import Meter
import torch
# from ..asg2cap.data.transforms import Int2Sent

params.greedy_or_beam = True


In [12]:
import os
model_root = '/root/.thexp/experiments/asg2cap.e0/asg2cap.trainers/0011.b632f246/modules'

model_pt = os.path.join(model_root,'keep.0000030.ckpt')

In [13]:
trainer.load_checkpoint(model_pt)

loading checkpoint | 
21-02-05 16:44:43 | loading model: model | 
21-02-05 16:44:43 | loading optimizers: optim | 
21-02-05 16:44:43 | loading other: 21-02-05 16:44:43 | loading vectors: {'Lall': '0.7316(0.6821)',
 'Lcap': '0.7316(0.6821)',
 'fn': '/root/.thexp/experiments/asg2cap.e0/asg2cap.trainers/0011.b632f246/modules/keep.0000030.ckpt'} | 


In [28]:
self = trainer
batch_data = batch

from thexp import Meter
from data.transforms import Int2Sent

meter = Meter()
int2sent_func = Int2Sent()


if params.greedy_or_beam:
    from thextra.inferences import sample_decode as decode_func
else:
    from thextra.inferences import beam_search_decode as decode_func

enc_outs = self.model.encode(batch_data=batch_data)  # type:dict
init_words = torch.zeros(batch_data['attn_masks'].size(0), dtype=torch.int64).to(self.device)

states = []
for _ in range(2):  # (hidden, cell)
    states.append(torch.zeros((2, params.batch_size, self.params.hidden_size),
                              dtype=torch.float32).to(self.device))

_, max_attn_len = batch_data['attn_masks'].shape
prev_attn_score = torch.zeros((params.batch_size, max_attn_len), device=self.device)
prev_attn_score[:, 0] = 1

enc_globals = enc_outs['init_states']
enc_memories = enc_outs['attn_fts']
enc_masks = batch_data['attn_masks']
flow_edges = batch_data['flow_edges']

result = decode_func(init_words, self.model.decoder.step_fn, params.max_words_in_sent,
                     greedy=True, states=states, enc_globals=enc_globals,
                     enc_memories=enc_memories, memory_masks=enc_masks,
                     prev_attn_score=prev_attn_score, flow_edges=flow_edges)

if params.greedy_or_beam:
    pred_sent, word_logprobs = result
    sent_pool = []
    for sent, word_logprob in zip(pred_sent, word_logprobs):
        sent_pool.append([(word_logprob.sum().item(), sent, word_logprob)])
else:
    sent_pool = result
    pred_sent = [pool[0][1] for pool in sent]

word_sents = []
gt_sents = []
for i, sent, gt_sent in zip(batch_data['index'], pred_sent, batch_data['caption_ids']):
    sent = sent.detach().cpu().numpy().tolist()
    gt_sent = gt_sent.detach().cpu().numpy().tolist()

    sent = int2sent_func(sent)
    gt_sent = int2sent_func(gt_sent)
    gt_sents.append(gt_sent)
    word_sents.append(sent)

gt_sents = {i: [v] for i, v in enumerate(gt_sents)}
word_sents = {i: [v] for i, v in enumerate(word_sents)}

self.acc_bleu_(gt_sents, word_sents, meter=meter)
self.acc_cider_(gt_sents, word_sents, meter=meter)

(1.7238018162070041, array([6.83237863e-01, 3.18185131e+00, 5.17484718e-02, 7.13428363e+00,
       8.33883776e-01, 3.05561714e-01, 4.62660104e-01, 7.86409468e-02,
       2.80662551e+00, 9.43192422e-01, 1.15172792e+00, 8.87019579e-01,
       0.00000000e+00, 1.65409347e-01, 4.75902650e-01, 9.19417308e-01,
       7.47406924e-01, 3.82002369e-01, 2.71265771e-01, 1.40136456e-01,
       2.40561105e-01, 1.31271006e-01, 3.03626584e-02, 3.21047357e+00,
       5.20694049e-01, 1.68328223e+00, 2.33831933e+00, 6.38795028e-04,
       1.19781988e-03, 7.23874857e-01, 4.48683638e-04, 1.64965453e-03,
       1.25503996e+00, 8.20668839e-01, 1.57341928e+00, 3.39825254e+00,
       6.10603517e-01, 2.25603177e+00, 4.41351208e+00, 6.15252939e+00,
       5.55018932e+00, 3.39923842e-01, 1.68002735e+00, 9.54076207e+00,
       4.66242914e+00, 1.78375771e+00, 3.59560486e-01, 4.01594241e-02,
       2.49685251e+00, 8.84389414e-01, 1.87855989e+00, 1.18551076e+00,
       1.95290753e-01, 2.11314051e+00, 1.78018592e-01, 2

In [59]:
from matplotlib import pyplot as plt

In [62]:
plt.imshow(plt.imread(img_path))

In [29]:
gt_sents[2]

['<BOS> a counter of cakes surrounded by a group of people']

In [30]:
word_sents[2]

['a table with a cake sitting on top of a wall in a room']